### Importing the libraries 

In [4]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os

In [5]:
# Loading environment variables from .env file
load_dotenv()

# Getting the API key from environment variables
api_key = os.getenv('IATI_API_KEY')

# API endpoint
url = 'https://api.iatistandard.org/datastore/activity/iati'

# parameters 
params = {
    'q': '*:*',  # Query all records
    'rows': 5 ,   # Limit the number of results
    'fq': 'iati_identifier:"CH-4-1980001679"',    # Filter for specific identifier
}

# headers
headers = {
    'Accept': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key 
}

# Making the GET request with parameters and headers
response = requests.get(url, params=params, headers=headers)

# response content for debugging
print(response.content)

if response.status_code == 200:
    try:
        # Parsing the XML response
        root = ET.fromstring(response.content)
        
        # Extracting data from XML
        activities = []
        for activity in root.findall('.//iati-activity'):
            activity_data = {
                'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
                'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
                'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
                'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
                'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
                'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
                'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
                'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
                'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
                'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
                'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
                'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
                'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
                'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
                'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
                'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
                'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
            }
            activities.append(activity_data)
        
        # Converting to DF
        df = pd.DataFrame(activities)
        display(df)
    except ET.ParseError as e:
        print("Error parsing XML:", e)
else:
    print(f"Error: {response.status_code}")


b'<?xml version="1.0" encoding="UTF-8"?><iati-activities generated-datetime="2024-07-22T18:39:51.557Z" version="2.03"><iati-activity last-updated-datetime="2024-05-31T16:47:52+00:00" xml:lang="en" default-currency="CHF" humanitarian="0" hierarchy="1" linked-data-uri="">\n  <iati-identifier>CH-FDJP-109458452-Trust-Fund-for-North-Africa</iati-identifier>\n  <reporting-org type="60" ref="CH-FDJP-109458452" secondary-reporter="0">\n    <narrative xml:lang="en">DCAF - Geneva Centre for Security Sector Governance</narrative>\n  </reporting-org>\n  <title>\n    <narrative xml:lang="en">DCAF Trust Fund for North Africa (TFNA)</narrative>\n  </title>\n  <description type="1">\n    <narrative xml:lang="en">The DCAF Trust Fund for North Africa (TFNA) is a pool funding instrument established in 2012 to finance DCAF supported security sector governance development goals, within a context of democratic transition, in countries of North Africa.</narrative>\n  </description>\n  <description type="2">\

,identifier,title,description,status,start_date,end_date,reporting_org,recipient_country,sector,budget,transaction_type,transaction_date,transaction_value,transaction_description,provider_org,receiver_org,finance_type
0,CH-FDJP-109458452-Trust-Fund-for-North-Africa,DCAF Trust Fund for North Africa (TFNA),The DCAF Trust Fund for North Africa (TFNA) is...,2,None,None,DCAF - Geneva Centre for Security Sector Gover...,None,The OECD DAC Guidelines on Security System Ref...,4121208,11,2020-03-01,2500000,NL contribution TFNA,DCAF,DCAF,110
1,XM-DAC-6-4-007279-01-6,ENVIRONMENTAL PROTECTION OF RADIKA RIVER BASIN...,SUSTAINABLE MANAGEMENT OF NATURAL RESOURCES OF...,4,2001-12-03,2023-04-07,AICS - Italian Agency for Cooperation and Deve...,None,Water resources conservation (including data c...,None,2,2001-11-26,20141.81,Commitment - Financial Year 2001,AICS - Italian Agency for Cooperation and Deve...,AICS - AGENZIA ITALIANA PER LA COOPERAZIONE AL...,None
2,XM-DAC-6-4-008877-01-3,INTEGRATION AND SUPPORT OF MINORITIES IN SOUTH...,INTEGRATION AND SUPPORT OF MINORITIES IN SOUTH...,4,2008-10-01,2021-12-31,AICS - Italian Agency for Cooperation and Deve...,None,Social Protection,None,2,2008-11-21,1645903.00,Commitment - Financial Year 2008,AICS - Italian Agency for Cooperation and Deve...,COSV - Coordinamento delle Organizzazioni per ...,None
3,XM-DAC-6-4-007337-05-0,SUPPORT TO THE SOCIAL EXPENDITURE ARGENTINA - ...,SUPPORT TO THE SOCIAL EXPENDITURE ARGENTINA - ...,2,2005-05-01,None,AICS - Italian Agency for Cooperation and Deve...,None,Basic health care,None,2,2005-08-05,264145.00,Commitment - Financial Year 2005,AICS - Italian Agency for Cooperation and Deve...,PAHO - Pan-American Health Organisation,None
4,XM-DAC-6-4-009093-01-1,TRAINING FOR THE LOCAL ECONOMIC DEVELOPMENT,PROMOTION SUITABLE POLICIES FOR THE LOCAL PROD...,4,2009-01-01,2023-04-07,AICS - Italian Agency for Cooperation and Deve...,None,Small and medium-sized enterprises (SME) devel...,None,2,2009-12-28,1543206.00,Commitment - Financial Year 2009,AICS - Italian Agency for Cooperation and Deve...,REGIONE PUGLIA,None


pagination, since even if i delete row limitation on the code above I get max 9 rows.

In [6]:
# Initializing variables for pagination
start = 0
rows = 100 
all_activities = []

while True:
  
    params = {
        'q': '*:*', 
        'start': start,
        'rows': rows
    }

    headers = {
        'Accept': 'application/json',
        'Ocp-Apim-Subscription-Key': api_key  
    }

    response = requests.get(url, params=params, headers=headers)

    print(response.content)

 
    if response.status_code == 200:
        try:
           
            root = ET.fromstring(response.content)
         
            activities = []
            for activity in root.findall('.//iati-activity'):
                activity_data = {
                    'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
                    'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
                    'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
                    'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
                    'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
                    'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
                    'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
                    'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
                    'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
                    'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
                    'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
                    'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
                    'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
                    'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
                    'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
                    'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
                    'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
                }
                activities.append(activity_data)
            
            if not activities:
                break  # No more activities to fetch, exit the loop

            all_activities.extend(activities)
            start += rows  # Move to the next batch of results

        except ET.ParseError as e:
            print("Error parsing XML:", e)
            break
    else:
        print(f"Error: {response.status_code}")
        break


df = pd.DataFrame(all_activities)



b'{ "statusCode": 429, "message": "Rate limit is exceeded. Try again in 1 seconds." }'
Error: 429
